## Импорт библиотек

In [ ]:
import warnings
from pathlib import Path

import torch
from torch import nn, optim

from utils import get_image_segmentation_model, Brain_MRI_Dataset, train_segmentation, test_segmentation, DiceLoss, FocalLoss, JaccardLoss

warnings.filterwarnings("ignore")

## Конфигурация

In [ ]:
# Устройство, на котором будут происходить все вычисления
device = "cuda" if torch.cuda.is_available() else "cpu"

# Пути до датасета
dataset_path = "D:/ProjectsData/Brain MRI segmentation/kaggle_3m"

# Имя базовой модели для классификатора изображений
image_model_type = "deeplabv3_resnet101"
# Функция потерь (BCEWithLogitsLoss, DiceLoss, FocalLoss, JaccardLoss)
criterion_name = "JaccardLoss"
# Название оптимизатора (Adam, AdamW)
optimizer_name = "AdamW"
# Количество предсказываемых классов
num_classes = 1
# Загрузка весов модели
pretrained = True
# Заморозка весов модели (кроме последних слоёв)
freeze_weight = False

# Количество обучающих эпох
num_epochs = 10
# Размер батча при обучении
batch_size = 12

# Пути сохранения и загрузки чекпоинта
save_path = "./deeplabv3_resnet101.pth"
checkpoint_path = "./deeplabv3_resnet101.pth"

## Инициализация необходимых переменных

In [ ]:
print(device)
if device == "cuda":
    print(torch.cuda.get_device_name())
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

# Список путей до изображений (исключаем маски)
path_array = list(Path(dataset_path).glob("**/*[!_mask].tif"))

# Инициализация модели и препроцессинга
model, preprocess = get_image_segmentation_model(name=image_model_type,
                                                 pretrained=pretrained,
                                                 freeze_weight=freeze_weight,
                                                 num_classes=num_classes)

# Инициализация датасета
dataset = Brain_MRI_Dataset(path_array=path_array, image_preprocess=preprocess, augmented=False, device=device)

# Инициализация функции потерь
if criterion_name == "BCEWithLogitsLoss":
    criterion = nn.BCEWithLogitsLoss()
elif criterion_name == "DiceLoss":
    criterion = DiceLoss()
elif criterion_name == "FocalLoss":
    criterion = FocalLoss()
elif criterion_name == "JaccardLoss":
    criterion = JaccardLoss()

# Инициализация оптимизатора
if optimizer_name == "Adam":
    optimizer = optim.Adam(model.parameters(), lr=0.001)
elif optimizer_name == "AdamW":
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)

# Загрузка чекпоинта
start_epoch = 0
current_pixelwise_accuracy = .0
if checkpoint_path:
    chekpoint = torch.load(checkpoint_path)
    model.load_state_dict(chekpoint["model"])
    criterion.load_state_dict(chekpoint["loss"])
    optimizer.load_state_dict(chekpoint["optimizer"])
    start_epoch = chekpoint['epoch'] - 1
    current_pixelwise_accuracy = chekpoint['pixelwise']
    print(f"Чекпоинт загружен\nСохраненная эпоха: {start_epoch}\nPixelwise Acc.: {current_pixelwise_accuracy}")

## Обучение

In [ ]:
train_segmentation(model=model, dataset=dataset, criterion=criterion, optimizer=optimizer, num_classes=num_classes,
                   batch_size=batch_size, num_epochs=num_epochs, start_epoch=start_epoch, save_path=save_path,
                   current_pixelwise_accuracy=current_pixelwise_accuracy, device=device)

## Тестирование

In [ ]:
chekpoint = torch.load(save_path)
model.load_state_dict(chekpoint["model"])

In [ ]:
image_path = "D:/ProjectsData/Brain MRI segmentation/kaggle_3m/TCGA_CS_6669_20020102/TCGA_CS_6669_20020102_11.tif"
pil_orig_image, pil_predict, pil_predict_masks = test_segmentation(model=model, image_preprocess=preprocess,
                                                                   image_path=image_path, proba_threshold=0.5, alpha=0.9,
                                                                   device=device)

In [ ]:
pil_orig_image

In [ ]:
pil_predict

In [ ]:
pil_predict_masks